In [1]:
    import importlib, pytest, abc, pathlib

In [2]:
    from importnb import Notebook

In [3]:
    def pytest_addoption(parser):
        group = parser.getgroup("general")
        group.addoption('--main', action='store_true',
                        help="Run in the main context.")

In [4]:
    class AlternativeModule(pytest.Module):
        def _getobj(self):
            return self.loader(self.parent.config.option.main and "__main__" or None).load(str(self.fspath))

In [5]:
    class NotebookModule(AlternativeModule): 
        loader = Notebook

In [6]:
    class AlternativeSourceText(abc.ABCMeta):
        def __call__(self, parent, path):
            for module in self.modules:
                if ''.join(pathlib.Path(str(path)).suffixes) in module.loader.extensions:
                    if not parent.session.isinitpath(path):
                        for pat in parent.config.getini('python_files'):
                            if path.fnmatch(pat.rstrip('.py') + path.ext): break
                        else: return
                    return module(path, parent)

In [7]:
    class NotebookTests(metaclass=AlternativeSourceText):
        modules = NotebookModule,

    pytest_collect_file = NotebookTests.__call__

In [8]:
    if __name__ ==  '__main__':
        from importnb.utils.export import export
        export('pytest_plugin.ipynb', '../../utils/pytest_plugin.py')